In [ ]:
# Conteo de valores únicos con pandas

In [ ]:
## Carga del Data Frame e importación de librerías

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/USUARIO/nsd_analysis_newspaper/main/corpus/notas.csv"
df = pd.read_csv(url)

In [ ]:
## Conteo de valores faltantes por columnas

In [ ]:
payment_missing = df['payment_method'].isna().sum()
city_missing = df['city'].isna().sum()
state_missing = df['state'].isna().sum()

print("payment_method missing:", payment_missing)
print("city missing:", city_missing)
print("state missing:", state_missing)

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/everpeak_retail.csv'

In [ ]:
## Fechas sospechosas y montos extremos

In [ ]:
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")

invalid_year_2026_count = (df['order_date'].dt.year==2026).sum()
missing_order_date_count = df['order_date'].isna().sum()
print("order_date año 2026:", invalid_year_2026_count)
print("order_date missing:", missing_order_date_count)

In [ ]:
## Cardinalidad de columnas clave

In [ ]:
customer_id_unicos = df['customer_id'].nunique()
payment_unicos =  df['payment_method'].nunique()
city_unicos =  df['city'].nunique()
state_unicos = df['state'].nunique()

print("customer_id nunique:", customer_id_unicos)
print("payment_method nunique:", payment_unicos)
print("city nunique:", city_unicos)
print("state nunique:", state_unicos)

In [ ]:
# Estudio de caso para selección de metodología

In [ ]:
## Detectar patrones de missingness (MAR/MNOAR/MCAR) pro grupos

In [ ]:
missing_city_by_pay = df['city'].isna().groupby(df['payment_method']).mean()
print(missing_city_by_pay)

In [ ]:
## Compara impacto entre drop e imputación

In [ ]:
before = df['order_value'].dropna().mean()
df["order_value_imputed"] = df['order_value'].fillna(df['order_value'].median())
after = df['order_value_imputed'].mean()

print(before)
print(after)

In [ ]:
## Calcular la media original, la media imputando mediana, y la media imputando media

In [ ]:
before = df['customer_age'].mean()

df["customer_age_med"] = df['customer_age'].fillna(df['customer_age'].median())
after_med = df['customer_age_med'].mean()

df["customer_age_mean"] = df['customer_age'].fillna(df ['customer_age'].mean())
after_mean = df['customer_age_mean'].mean()

print(before)
print(after_med)
print(after_mean)

In [ ]:
# Análisis con funciones for personalizadas

In [ ]:
## Convertir a numértico múltiples columnas

In [ ]:
def convertir_columnas_numericas(df, columnas):
    for col in columnas:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

columnas_numericas = ["price", "quantity", "order_value", "customer_id"]

df = convertir_columnas_numericas(df, columnas_numericas)

df.info()

In [ ]:
## Limpiar múltiples columnas de texto con un loop

In [ ]:
def step_strip_text(df):
    columnas = ['product_category', 'city', 'state']

    for col in columnas:
        df[col]=df[col].str.strip()

    return df

df = step_strip_text(df)
print(df.head())

In [ ]:
# Pipeline completa de limpieza de datos

In [ ]:
## Función para reemplazar sentinels

In [ ]:
def reemplazar_sentinels(df, sentinels, numeric_cols):
    for col in numeric_cols:
        df[col]=df[col].replace(sentinels, pd.NA)
    return df

print("Valores ausentes iniciales:")
print(df[["customer_age", "price"]].isna().sum())

valores_erroneos = [-999, 999, 0, -1]
columnas_numericas = ['customer_age', 'price']

df = reemplazar_sentinels(df, valores_erroneos, columnas_numericas)
print("\nValores ausentes después:")
print(df[["customer_age", "price"]].isna().sum())

In [ ]:
## Función para rellenar ausentes

In [ ]:

def rellenar_ausentes(df, cols_fill):
    for col in cols_fill:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(df[col].mean(), inplace=True)
    return df

In [ ]:
## Función pipeline

In [ ]:
def limpiar_df(df):
    valores_erroneos = [-999, 999, 0, 1]
    columnas_numericas = ['customer_age', 'price']

    df= reemplazar_sentinels(df, valores_erroneos, columnas_numericas)
    df= rellenar_ausentes(df, columnas_numericas)

    return df

In [ ]:
# Visualización

In [ ]:
## Resumen numérico

In [ ]:
df_fashion = df[df['product_category']=='Fashion']
df_sports = df[df['product_category']=='Sports']

columnas_numericas = ['order_value', 'customer_age', 'price', 'quantity']

print('Resumen estadístico de la categoría Fashion')
print(df_fashion[columnas_numericas].describe())
print()
print('Resumen estadístico de la categoría Sports')
print(df_sports[columnas_numericas].describe())

In [ ]:
## Promedio vs mediana del gasto

In [ ]:
df_grocery = df[df['product_category']=='Grocery']

promedio = df_grocery['order_value'].mean()
mediana = df_grocery['order_value'].median()

print("Promedio del gasto en Grocery:", promedio)
print("Mediana del gasto en Grocery:", mediana)

print("El promedio está afectado por outliers o valores atípicos en Grocery.")

In [ ]:
## Promedio vs mediana

In [ ]:
print("Promedio de quantity: ", df['quantity'].mean())
print("Mediana de quantity: ", df['quantity'].median())
print("El promedio está afectado por los outliers o valores atípicos.")

In [ ]:
## Resumen numérico por ciudad

In [ ]:
df_ny = df[df['city']=='New York']
df_la = df[df['city']=='Los Angeles']

columnas_numericas = ['order_value', 'customer_age', 'price', 'quantity']

print('Resumen estadístico de la ciudad New York')
print(df_ny.describe())
print()
print('Resumen estadístico de la ciudad Los Angeles')
print(df_la.describe())

In [ ]:
## Análisis categórico

In [ ]:
columnas_categoricas = ['payment_method', 'product_category']

df_ny = df[df['city']=='New York']
df_chicago = df[df['city']=='Chicago']

print("Resumen categórico - New York")
print(df_ny.describe())
print()
print("Resumen categórico - Chicago")
print(df_chicago.describe())

In [ ]:
## Distribución completa de categorías con value_counts

In [ ]:
columnas_categoricas = ['product_category', 'payment_method', 'city', 'state']

for col in columnas_categoricas:
    print(col)
    print("Frecuencia absoluta")
    print(df[col].value_counts())
    print("Frecuencia relativa")
    print(df[col].value_counts(normalize=True))
    print() # mantén salto de línea

In [ ]:
# Visualización de datos

In [ ]:
## Análisis de la distribución de datos

In [ ]:
import matplotlib.pyplot as plt

counts, bin_edges, _= plt.hist(df['price'], bins=10, range=(0,1000), color='skyblue', edgecolor='black')

plt.xticks(bin_edges)
plt.xlabel('Precio')
plt.ylabel('Cantidad')
plt.title('Distribución de Precios')
plt.show()

In [ ]:
## Distribución de edades de clientes

In [ ]:
counts, bin_edges, _ = plt.hist(df['customer_age'], bins=10, color='skyblue', edgecolor='black')

plt.xticks(bin_edges)

plt.xlabel('Edades de los Clientes')
plt.ylabel('Cantidad')
plt.title('Distribución de Edades')
plt.show()

In [ ]:
Boxplot del valor total de pedidos

In [ ]:
df_fashion = df[df['product_category']=='Fashion']
df_sports = df[df['product_category']=='Sports']

sns.boxplot(df_fashion['order_value'], color='skyblue')
plt.xlabel('Gasto del Cliente')
plt.title('Boxplot de Gastos - Categoría Fashion')
plt.show()

sns.boxplot(df_sports['order_value'], color='skyblue')
plt.xlabel('Gasto del cliente')
plt.title('Boxplot de Gastos - Categoría Sports')
plt.show()

In [ ]:
## Detectar outlier con IQR

In [ ]:
Q1 = df['order_value'].quantile(0.25)
Q3 = df['order_value'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5*IQR
limite_superior = Q3 + 1.5*IQR

print('Primer cuartil: ', Q1)
print('Tercer cuartil: ', Q3)
print('IQR: ', IQR)

print("\nRegistros abajo del límite inferior")
outilers_inferiores= df[df['order_value']<limite_inferior]
print(outilers_inferiores)

print("\nRegistros arriba del límite superior")
outliers_superiores= df[df['order_value']>limite_superior]
print(outliers_superiores)

In [ ]:
## Z-scores

In [ ]:
mean = df['order_value'].mean()

#cálculo de la desviación estándar
std = df['order_value'].std()

#Crea el z score
df['z'] = (df['order_value']-mean)/std

#Calcula los valores extremos
valores_extremos = df[df['z']>3]
print(valores_extremos)

In [ ]:
## Comparar métodos en price

In [ ]:
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

mean = df['price'].mean()
std = df['price'].std()
df['z'] = (df['price']-mean)/std

print('Outliers usando IQR:')
outliers_iqr = df[(df['price']<lower) | (df['price']>upper)]
print(outliers_iqr)

print('\nOutliers usando Z-Score:')
outliers_z = df[df['z']>3]
print(outliers_z)

In [ ]:
# Segmentación

In [ ]:
cantidad_promedio = df['quantity'].mean()
cantidad_mediana = df['quantity'].median()
print("Promedio:", cantidad_promedio)
print("Mediana:",cantidad_mediana)
print()

## Segmentación con media o promedio
if cantidad_promedio > 22:
	print("En promedio: volumen alto")
elif cantidad_promedio >= 10:
    print(f'En promedio: volumen medio')
else:
	print("En promedio: volumen bajo")

## Segmentación con mediana
if cantidad_mediana > 22:
	print("Según la mediana: volumen alto")
elif cantidad_mediana >= 10:
    print(f'Según la mediana: volumen medio')
else:
	print("Según la mediana: volumen bajo")

In [ ]:
# Segmentación avanzada con función por edad y volúmen de compra

In [ ]:
def classify_volume(row):
    age = row['customer_age']
    qty = row['quantity']

    if pd.isna(age) or pd.isna(qty):
        return "Error en Datos"

    if qty > 22:
        if age > 55:
            return "Sr. High Volume"
        else:
            return "Jr. High Volume"

    elif qty <= 22:
        if age > 55:
            return "Sr. Low Volume"
        else:
            return "Jr. Low Volume"

df["volume_segment"] = df.apply(classify_volume, axis=1)
print(df['volume_segment'].value_counts())

In [ ]:
## Segmentación avanzada con función por método de pago y volúmen de compra

In [ ]:
def classify_payment(row):
    card = row['payment_method']
    qty = row['quantity']

    if pd.isna(card) or pd.isna(qty):
        return "Error en Datos"

    if qty > 22:
        if card == 'credit_card' or card == 'debit_card':
            return 'card_high_volume'
        else:
            return 'no_card_high_volume'

    elif qty <= 22:
        if card == 'credit_card' or card =='debit_card':
            return 'card_low_volume'
        else:
            return 'no_card_low_volume'

df['payment_segment'] = df.apply(classify_payment, axis=1)
print(df['payment_segment'].value_counts())